In [63]:
from pathlib import Path
from kedro.framework.session import KedroSession
from kedro.framework.startup import bootstrap_project
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import great_expectations as gx
from great_expectations.profile.user_configurable_profiler import UserConfigurableProfiler
from great_expectations.data_context import DataContext
from great_expectations.core.expectation_configuration import ExpectationConfiguration

# Get the project root (2 levels up from this script)
project_path = Path.cwd().parents[1]

# Bootstrap project (loads pyproject.toml and settings)
bootstrap_project(project_path)

# Create Kedro session for your project
session = KedroSession.create()

# Load the Kedro context (access pipelines, catalog, etc.)
context = session.load_context()

# Now you can load your dataset from the catalog:
df = context.catalog.load("loans")

[06/17/25 14:44:16] WARNING  c:\Users\35191\anaconda3\anaconda\envs\Machine_Learning_Operations\Lib ]8;id=339305;file://c:\Users\35191\anaconda3\anaconda\envs\Machine_Learning_Operations\Lib\warnings.py\warnings.py]8;;\:]8;id=564170;file://c:\Users\35191\anaconda3\anaconda\envs\Machine_Learning_Operations\Lib\warnings.py#110\110]8;;\
                             \site-packages\kedro\io\data_catalog.py:165: KedroDeprecationWarning:                 
                             `DataCatalog` has been deprecated and will be replaced by                             
                             `KedroDataCatalog`, in Kedro 1.0.0.Currently some `KedroDataCatalog`                  
                             APIs have been retained for compatibility with `DataCatalog`,                         
                             including the `datasets` property and the `get_datasets`,                             
                             `_get_datasets`, `add`,` list`, `add_feed_dict`, and `shallow_copy`                   
                             methods. These will be removed or replaced with updated alternatives                  
                             in Kedro 1.0.0. For more details, refer to the documentation:                         
                             https://docs.kedro.org/en/stable/data/index.html#kedrodatacatalog-expe                
                             rimental-feature                                                                      
                               warnings.warn(                                                                      
                                                                                                                   

[06/17/25 14:44:17] INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=638685;file://c:\Users\35191\anaconda3\anaconda\envs\Machine_Learning_Operations\Lib\site-packages\kedro_mlflow\framework\hooks\mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=617230;file://c:\Users\35191\anaconda3\anaconda\envs\Machine_Learning_Operations\Lib\site-packages\kedro_mlflow\framework\hooks\mlflow_hook.py#65\65]8;;\

                    WARNING  No 'mlflow.yml' config file found in environment. Default            ]8;id=667549;file://c:\Users\35191\anaconda3\anaconda\envs\Machine_Learning_Operations\Lib\site-packages\kedro_mlflow\framework\hooks\mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=248211;file://c:\Users\35191\anaconda3\anaconda\envs\Machine_Learning_Operations\Lib\site-packages\kedro_mlflow\framework\hooks\mlflow_hook.py#78\78]8;;\
                             configuration will be used. Use ``kedro mlflow init`` command in CLI                  
                             to customize the configuration.                                                       

                    WARNING  c:\Users\35191\anaconda3\anaconda\envs\Machine_Learning_Operations\Lib ]8;id=808287;file://c:\Users\35191\anaconda3\anaconda\envs\Machine_Learning_Operations\Lib\warnings.py\warnings.py]8;;\:]8;id=154484;file://c:\Users\35191\anaconda3\anaconda\envs\Machine_Learning_Operations\Lib\warnings.py#110\110]8;;\
                             \site-packages\kedro\io\core.py:355: KedroDeprecationWarning: The                     
                             MatplotlibWriter class has been renamed to MatplotlibDataset. The                     
                             MatplotlibWriter name is deprecated and will be removed in a future                   
                             release.                                                                              
                               init_func(self, *args, **kwargs)                                                    
                                                                                                                   

                    INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=484054;file://c:\Users\35191\anaconda3\anaconda\envs\Machine_Learning_Operations\Lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=971220;file://c:\Users\35191\anaconda3\anaconda\envs\Machine_Learning_Operations\Lib\site-packages\kedro_telemetry\plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[06/17/25 14:44:18] INFO     Loading data from loans (CSVDataset)...                            ]8;id=853116;file://c:\Users\35191\anaconda3\anaconda\envs\Machine_Learning_Operations\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=797818;file://c:\Users\35191\anaconda3\anaconda\envs\Machine_Learning_Operations\Lib\site-packages\kedro\io\data_catalog.py#401\401]8;;\

In [64]:
df.columns = df.columns.str.strip()

# Split 70% for reference, 30% for analysis
df_ref = df.sample(frac=0.7, random_state=42)
df_ana = df.drop(df_ref.index)

In [65]:
context = gx.get_context(context_root_dir = "gx")

In [66]:
# Register a pandas-based datasource
datasource_name = "example" 
try:
    datasource = context.sources.add_pandas(datasource_name)
except:
    print("Data Source already exists.")
    datasource = context.datasources[datasource_name]

Data Source already exists.


In [67]:
context


{
  "anonymous_usage_statistics": {
    "explicit_id": true,
    "explicit_url": false,
    "usage_statistics_url": "https://stats.greatexpectations.io/great_expectations/v1/usage_statistics",
    "enabled": true,
    "data_context_id": "62b31bc3-528b-4a42-8087-dddee0817143"
  },
  "checkpoint_store_name": "checkpoint_store",
  "config_variables_file_path": "uncommitted/config_variables.yml",
  "config_version": 3.0,
  "data_docs_sites": {
    "local_site": {
      "class_name": "SiteBuilder",
      "show_how_to_buttons": true,
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "uncommitted/data_docs/local_site/"
      },
      "site_index_builder": {
        "class_name": "DefaultSiteIndexBuilder"
      }
    }
  },
  "datasources": {},
  "evaluation_parameter_store_name": "evaluation_parameter_store",
  "expectations_store_name": "expectations_store",
  "fluent_datasources": {
    "example": {
      "type": "pandas"
    }
  },
  "

In [68]:
print("data source", context.list_datasources())

data source [{'type': 'pandas', 'name': 'example'}]


In [69]:
print(datasource)

name: example
type: pandas



In [70]:
suite_bank = context.add_or_update_expectation_suite(expectation_suite_name="example_suite")

In [71]:
suite_bank


{
  "expectation_suite_name": "example_suite",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.12"
  }
}

In [72]:
df_ref.columns


Index(['loan_id', 'no_of_dependents', 'education', 'self_employed',
       'income_annum', 'loan_amount', 'loan_term', 'cibil_score',
       'residential_assets_value', 'commercial_assets_value',
       'luxury_assets_value', 'bank_asset_value', 'loan_status'],
      dtype='object')

In [73]:
df_ref["education"].describe()


count          2988
unique            2
top        Graduate
freq           1498
Name: education, dtype: object

In [74]:
df_ref["education"].unique()

array([' Graduate', ' Not Graduate'], dtype=object)

In [75]:
df_ref['education'] = df_ref['education'].str.strip()
df_ref["education"].unique()

array(['Graduate', 'Not Graduate'], dtype=object)

In [76]:
expectation_education = ExpectationConfiguration(
    expectation_type="expect_column_distinct_values_to_be_in_set",
    kwargs={
        "column": "education",
        "value_set" : ['Graduate', 'Not Graduate']
    },
)
suite_bank.add_expectation(expectation_configuration=expectation_education)

{"expectation_type": "expect_column_distinct_values_to_be_in_set", "kwargs": {"column": "education", "value_set": ["Graduate", "Not Graduate"]}, "meta": {}}

In [77]:
suite_bank


{
  "expectation_suite_name": "example_suite",
  "ge_cloud_id": null,
  "expectations": [
    {
      "expectation_type": "expect_column_distinct_values_to_be_in_set",
      "kwargs": {
        "column": "education",
        "value_set": [
          "Graduate",
          "Not Graduate"
        ]
      },
      "meta": {}
    }
  ],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.12"
  }
}

In [78]:
context.add_or_update_expectation_suite(expectation_suite=suite_bank)


{
  "expectation_suite_name": "example_suite",
  "ge_cloud_id": null,
  "expectations": [
    {
      "expectation_type": "expect_column_distinct_values_to_be_in_set",
      "kwargs": {
        "column": "education",
        "value_set": [
          "Graduate",
          "Not Graduate"
        ]
      },
      "meta": {}
    }
  ],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.12"
  }
}